In [1]:
import sys
import numpy as np
from numba import jit
sys.path.append("../")
from src.utils import GenOverlapGroup
from src.regularizer import OGL1

In [2]:
p = 7
generator = GenOverlapGroup(p, 3, 5)
starts, ends = generator.get_group()
index = [*range(p)]
for i in range(len(starts)):
    print(index[starts[i]:ends[i]+1])

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]


In [3]:
starts, ends

([0, 1, 2], [4, 5, 6])

In [6]:
r = OGL1(Lambda=1, dim=p, starts=starts, ends=ends)
X = np.array([1.0, 4.2, 9.3, 12.6, 15.9, 8.4, 7.0]).reshape(-1,1)
print(r.func_ub(X,approx=1))
print(r.func_ub(X.copy(), approx=2))
print(r.func_lb(X))

61.79562331502589
85.26358866489885
56.40301410385796


In [7]:
r.Lambda_group

array([2.23606798, 2.23606798, 2.23606798])

In [15]:
p = int(1e4); ngrp=200; grp_size=200
generator = GenOverlapGroup(p, ngrp, grp_size)
starts, ends = generator.get_group()
r = OGL1(Lambda=1, dim=p, starts=starts, ends=ends)
np.random.seed(10)
X = np.random.randn(p,1)
print(r.func_ub(X, approx=1))
print(r.func_ub(X.copy(), approx=2))
print(r.func_lb(X))

9912.433804364251
19485.532211739825
1399.614867848165


In [ ]:
import numpy as np
from numpy.linalg import norm
a = np.array([2,3]); b = np.array([0,1])
print(norm(a) + norm(b))
a = np.array([2,0]); b = np.array([3,1])
print(norm(a) + norm(b))
a = np.array([2,3/2]); b = np.array([3/2,1])
print(norm(a) + norm(b))
print("===")
print(norm(np.array([2,3,1])))

In [25]:
import numpy as np
from numpy.linalg import norm
a = np.array([2,3]); b = np.array([0,1])
print(norm(a) + norm(b))
a = np.array([2,0]); b = np.array([3,1])
print(norm(a) + norm(b))
a = np.array([2,3/2]); b = np.array([3/2,1])
print(norm(a) + norm(b))
print("===")
print(norm(np.array([2,3,1])))

4.60555127546399
5.16227766016838
4.302775637731995
===
3.7416573867739413


In [13]:
from scipy.optimize import minimize_scalar
def f(x):
    return np.sqrt(4+x**2) + np.sqrt((3-x)**2 + 1)
res = minimize_scalar(f)
res.x

2.000000045899457

In [12]:
a = np.array([2,2]); b = np.array([1,1])
print(norm(a) + norm(b))

4.242640687119286


In [63]:
p = 7
starts, ends = [0, 3, 5], [2, 4, 6]
r = OGL1(Lambda=20, dim=p, starts=starts, ends=ends)
xk = 1.0* np.array([1, 2, 3, 4, 5, 6, 7]).reshape(-1, 1)
gradfxk = 0.1 * np.array([1, 2, 3, 4, 5, 6, 7]).reshape(-1, 1)
alphak = 0.2
uk = xk - alphak * gradfxk

proximal = np.zeros_like(xk)
for i in range(len(starts)):
    start, end = r.starts[i], r.ends[i]
    ukg = uk[start:end]
    ukg_norm = np.sqrt(np.dot(ukg.T, ukg))[0][0]
    if ukg_norm != 0:
        temp = 1 - ((r.Lambda_group[i] * alphak) / ukg_norm)
    else:
        temp = -1
    proximal[start:end] = max(temp, 0) * ukg
proximal.T

array([[0.        , 0.        , 0.        , 0.38619117, 0.48273896,
        2.19856805, 2.56499606]])

In [ ]:
0.517089950113724   1.034179900227449   1.551269850341173   3.036547791401228   3.795684739251535   4.959642013383156 5.786249015613682


In [50]:
temp = (proximal - uk) / alphak
dual_norm = r.dual(temp)
y = min(1, 1 / dual_norm) * temp
temp2 = proximal - uk
rproximal = 0
for i in range(3):
    start, end = r.starts[i], r.ends[i]
    proximalg = proximal[start:end]
    temp3 = np.sqrt(np.sum(proximalg*proximalg))
    rproximal += r.Lambda_group[i] * temp3
primal = np.dot(temp2.T, temp2)[0][0] / (2 * alphak) + rproximal
dual_negative = ((alphak / 2) * (np.dot(y.T, y)) + np.dot(uk.T, y))[0][0]
primal + dual_negative

0.0

In [52]:
temp = (proximal - uk) / alphak
dual_norm = r.dual(temp)
y = min(1, 1 / dual_norm) * temp
temp2 = proximal - uk
primal = np.dot(temp2.T, temp2)[0][0] / (2 * alphak) + r.func_ub(proximal)
dual_negative = ((alphak / 2) * (np.dot(y.T, y)) + np.dot(uk.T, y))[0][0]
primal + dual_negative

0.0

In [62]:
vk = np.zeros_like(uk)
for i in range(3):
    start, end = r.starts[i], r.ends[i]
    ukg = uk[start:end]
    ukg_norm = np.sqrt(np.sum(ukg*ukg))
    vk[start:end] = (r.Lambda_group[i]* alphak/ukg_norm) * ukg
(uk-vk).T

array([[0.51708995, 1.0341799 , 1.55126985, 3.03654779, 3.79568474,
        4.95964201, 5.78624902]])

In [60]:
vk.T

array([[2.31455025, 4.6291005 , 6.94365075, 4.41726104, 5.5215763 ,
        4.60178993, 5.36875492]])

In [61]:
(uk - proximal).T

array([[0.46291005, 0.9258201 , 1.38873015, 0.88345221, 1.10431526,
        0.92035799, 1.07375098]])

In [1]:
import numpy as np
from numpy.linalg import norm

In [20]:
def check_feasibility(z, starts, ends, bounds):
    for i in range(len(starts)):
        start, end = starts[i], ends[i]
        zg = z[start:end]
        zg_norm = np.sqrt(np.sum(zg*zg))
        if zg_norm > bounds[i]:
            z[start:end] = (bounds[i]/zg_norm) * zg
    return z


In [24]:
z = np.array([3.0,4.0,5.0])
starts = [0,1]
ends = [2,3]
check_feasibility(z, starts, ends, [5, 2])

array([3.        , 1.2493901 , 1.56173762])

In [18]:
class GenOverlapGroup:
    def __init__(self, dim, num_grp=None, grp_size=None, overlap_ratio=None):
        self.dim = dim
        self.num_grp = num_grp
        self.grp_size = grp_size
        self.overlap_ratio = overlap_ratio

    def get_group(self):
        if self.num_grp is not None and self.grp_size is not None:
            if self.grp_size * self.num_grp <= self.dim:
                raise ValueError("grp_size is too small to have overlapping.")
            if self.grp_size >= self.dim:
                raise ValueError("grp_size is too large that each group has all variables.")
            exceed = self.num_grp * self.grp_size - self.dim
            overlap_per_group = int(exceed / (self.num_grp - 1))
            starts = [0] * self.num_grp
            ends = [0] * self.num_grp
            for i in range(self.num_grp):
                if i == 0:
                    start = 0
                    end = start + self.grp_size - 1
                else:
                    start = end - overlap_per_group + 1
                    end = min(start + self.grp_size - 1, self.dim - 1)
                    if start == starts[i - 1] and end == ends[i - 1]:
                        self.num_grp = i
                        print(f"The actual number of group is {self.num_grp}")
                        break
                starts[i] = start
                ends[i] = end
            return starts[:i + 1], ends[:i + 1]
        elif self.grp_size is not None and self.overlap_ratio is not None:
            if self.grp_size >= self.dim:
                raise ValueError("grp_size is too large that each group has all variables.")
            overlap = int(self.grp_size * self.overlap_ratio)
            if overlap < 1:
                msg = "current config of grp_size and overlap_ratio cannot produce overlapping groups.\n"
                msg += "overlap_ratio is adjusted to have at least one overlap."
                warnings.warn(msg)
                overlap = 1
            starts = []
            ends = []
            self.num_grp = 0
            start = 0
            end = self.grp_size - 1
            while True:
                print(start, end)
                starts.append(start)
                ends.append(end)
                self.num_grp += 1
                # update
                start = end - (overlap - 1)
                end = min(start + self.grp_size - 1, self.dim - 1)
                if end == ends[-1]:
                    break
            
            return starts, ends
        else:
            raise ValueError("check your inputs!")

In [20]:
gen = GenOverlapGroup(100, grp_size=30, overlap_ratio=0.8)
gen.get_group()

0 29
6 35
12 41
18 47
24 53
30 59
36 65
42 71
48 77
54 83
60 89
66 95
72 99


([0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72],
 [29, 35, 41, 47, 53, 59, 65, 71, 77, 83, 89, 95, 99])

In [13]:
import sys
import numpy as np
from numba import jit
sys.path.append("../")
from src.utils import GenOverlapGroup

In [36]:
class natOG:
    def __init__(self, Lambda, dim, starts, ends):
        """
        Lambda: scalar > 0
        starts: a list of numbers speficy the starting index of each group
          ends: a list of numbers speficy the end index of each group

        For example, a overlapping group configuration, the number stands for
        the index of the variable.
        {g0=[0,1,2,3,4],g1=[3,4,5,6,7], g2=[5,6,7,8,9]}
        stars = [0, 3, 5]
        ends  = [4, 7, 9]
        """
        self.p = dim
        self.Lambda = Lambda
        self.K = len(starts)
        # a np.array that stores the number of group that each coordinate belongs to
        # self.freq = np.zeros((self.p, 1))
        self.group_size = np.zeros(self.K, dtype=np.int64)
        self.groups = {}
        for i in range(self.K):
            # self.freq[starts[i]:ends[i] + 1] += 1
            self.group_size[i] = ends[i] - starts[i] + 1
            self.groups[i] = np.arange(starts[i], ends[i] + 1)
        self.Lambda_group = Lambda * np.sqrt(self.group_size)
        self.starts = np.array(starts)
        # since python `start:end` will include `start` and exclude `end`,
        # we add 1 to the `end` so the G_i-th block of X is indexed by X[start:end]
        self.ends = np.array(ends) + 1

    def __str__(self):
        return("Natural Overlapping Group L1")

    def func(self, X):
        return _natf(X, self.starts, self.ends, self.Lambda_group)

    def createYStartsEnds(self):
        self.Ystarts = [0] * self.K
        self.Yends = [0] * self.K
        start = 0
        for i in range(self.K):
            end = start + self.group_size[i] - 1
            self.Ystarts[i] = start
            self.Yends[i] = end + 1
            start = end + 1

In [37]:
p = 30
grp_size = 5
overlap_ratio = 0.2
generator = GenOverlapGroup(p, grp_size=grp_size,overlap_ratio=overlap_ratio)
starts, ends = generator.get_group()
for i in range(len(starts)):
    print(starts[i], ends[i])

0 4
4 8
8 12
12 16
16 20
20 24
24 28
28 29


In [38]:
r = natOG(1, p, starts, ends)
r.createYStartsEnds()

In [44]:
for i in range(r.K):
    print(r.starts[i], r.ends[i])
    print(r.Ystarts[i], r.Yends[i])
    print("====")

0 5
0 5
====
4 9
5 10
====
8 13
10 15
====
12 17
15 20
====
16 21
20 25
====
20 25
25 30
====
24 29
30 35
====
28 30
35 37
====


In [43]:
r.ends[1]- r.starts[1],  r.Yends[1]- r.Ystarts[1]

(5, 5)

In [35]:
def changeU(u):
    u[0] = 2
    return u
u = np.array([1,100,-1])
changeU(u), u

(array([  2, 100,  -1]), array([  2, 100,  -1]))

In [45]:
np.array([0.0,1.0,0.0]) == 0.0, np.full(4, True)

(array([ True, False,  True]), array([ True,  True,  True,  True]))

In [100]:
from scipy.sparse import csc_matrix

row = np.array([0, 2, 2, 0, 1, 2,3,3])
col = np.array([0, 0, 1, 2, 2, 2,0,1])
data = np.array([3, 4, 1, 2, 6, 7,9,1])
Y = csc_matrix((data, (row, col)), shape=(4, 3))
print(Y.toarray())
row_sub = np.arange(4)[np.array([True, False, True, False]).reshape(-1,1).reshape(-1)].reshape(-1,1)
col_sub = np.array([False, True, True])
Ysub = Y[row_sub, col_sub]
print(Ysub.toarray())
Y[row_sub, col_sub] = csc_matrix((2,2))
print(Y.toarray())
# indptr = np.array([0, 2, 3, 6])
# indices = np.array([0, 2, 2, 0, 1, 2])
# data = np.array([1, 2, 3, 4, 5, 6])
# csc_matrix((data, indices, indptr), shape=(3, 3)).toarray()


[[3 0 2]
 [0 0 6]
 [4 1 7]
 [9 1 0]]
[[0 2]
 [1 7]]
[[3 0 2]
 [0 0 6]
 [4 1 7]
 [9 1 0]]


In [86]:
A = np.arange(24).reshape(4,6)
M=csc_matrix(A)
A[[[1],[2]],[1,2,3]]

array([[ 7,  8,  9],
       [13, 14, 15]])

In [89]:
M[[[1],[2]],[1,2,3]].toarray()

array([[ 7,  8,  9],
       [13, 14, 15]], dtype=int64)

In [101]:
M.A

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23]], dtype=int64)

In [10]:
from scipy.sparse import csc_matrix
import numpy as np

In [39]:
def create_I(starts, ends, GA):
    indices = []
    indptr = [0]
    num_elements = 0
    for j in range(GA):
        num_elements +=  ends[j] - starts[j]
        indices += list(range(starts[j],ends[j]))
        indptr.append(num_elements)
    indices = np.array(indices)
    indptr = np.array(indptr)
    data = np.ones_like(indices)
    return data, indices, indptr

In [40]:
starts, ends, GA = [0,3], [5,7], 2
data, indices, indptr = create_I(starts, ends, GA)
I = csc_matrix((data, indices, indptr), shape=(8, 2))
I.toarray()

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [0, 0]])

In [38]:
data, indices, indptr

(array([1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([0, 1, 2, 3, 4, 3, 4, 5, 6]),
 array([0, 5, 4]))

In [ ]:
I = np.zeros((self.p, GA))
# set up the indicator functio 1{j\in g}
for j in range(GA):
    I[starts[j]:ends[j], j] = 1